# Brief Paper explaining design choices

# Multi-Agent RAG system
This Jupyter Notebook serves as a proof of concept of a multi-agent solution for MedTech regulations. In order, you'll find a brief paper explaining design choices, the README to run the code, and the code itself divided into cells.

The technical intent of the system is to provide clear answers to questions on WHO and FDA documentations on medical devices, using a 3-to-4 agents system composed like this:

- an LLM orchestrator that receives the question and coordinates agents;
- a RAG agent capable of retrieving documents related to the question;
- an LLM response agent to put together the answer based on the documents retrieved and the question;
- a possible fourth agent to be decided (chunking agent, source-verifier agent, compare agent, prompt agent to improve prompts, etc.).

## Design Choices

The major constraints were time and no use of AI or other multi-agentic solutions, therefore I defaulted to a working prototype of a multi-agent RAG using standards methods and libraries.

### 1. RAG Implementation

I am familiar with ChromaDB, therefore I went with it. 

#### 1.1 Chunking Logic

Initially, I built the simplest possible RAG, with a chunking logic based on a fixed character length. Then I had a few different options:

- Paragraph logic: divide documents into paragraph and ignore short ones. **In the end I used this one as it is not the optimal solution but is the best one I was able to implement in the time I had**;
- Text conversion: converted PDFs into txt and tried a few different ideas. Unexpectedly, txt conversion worked worse than PDF;
- Semantic chunking: find a way to create logically coherent paragraph using embeddings and similarity scores. I didn't have the time to implement this logic from scratch;
- Agentic chunking: create a new AI agent to handle this task. I tried it and it worked pretty well but I got stuck in parsing and ran out of time.

#### 1.2 Embedding Function

I decided to use the deafult Chroma library (all-MiniLM-L6-v2) for ease of use, cost and efficiency. In production I probably would evaluate the trade offs of the openai embedding function.

#### 1.3 Other Considerations

I decided not to go with multi-hop retrieval, as it was too complicated for the simple chunking logic I had. I think that with semantic/agentic chunking, a multi-hop retrieval with (possibly) a retrieval agent could give good results.

### 2. Prompting

#### 2.1 Prompting Structure

In the prompt I followed this structure:

- Clear role
- Explicit tasks
- Output formatting
- Contextual information
- Additional Guidelines and edge cases

The response agent also got a persona, since it is the agent that communicates with the users. For this particular case, I decided not to use any fancy technique (chain of thought, tree of thought, self-consistency, etc.), but to provide (when useful) one example (so one-shot prompting).

#### 2.2 Output Parameters and Structure

I used the built-in generation-config to set the parameters for the LLM response. I set the temperature to 0.2 (low since I want it to be repeatable and reliable), top k and p to the lower end of the standard range, and the max output tokens to 8192 (actually an unimportant parameter for 1.5-flash, since it is already concise enough).

The output of the orchestrator needs to be standardized in order to be fed to the agents. I chose a JSON format like {"agent_to_call":"", "output": "", "relevant_info":""}, and it worked pretty well. I also tried to use the outputschema from generation_config but it worked worse. I think it probably is a better option but needs more fine-tuning (like Pydantic).

### 3. Agent Orchestration

Since I have a premium subscription, I chose Gemini as LLM. Among the different models, the ones who showed better behaviour were 1.5-flash and 1.5 pro. In the end I opted for 1.5-flash as it increases speed and efficiency without losing much precision.

#### 3.1 Handling Data Flow between Agents

In a multi-agent system, it is crucial to standardize the inputs and outputs of the agents, since you'll wrap things in a loop that will execute tasks and you won't be able to scale things if you have custom I/O for your agents. I chose to apply a brute-force workflow, where every agent has its own memory and it gets shared throughout the execution of tasks, serving as a context for both the orchestrator and the other agents. With more time, I think the best option (always assuming no pre-made agentic solutions) would be to create a 'state' object that stores information and that agents can access to gather the info they need and update it before passing the control to the next one. Additionally, an issue with this type of memory is that all agents have access to everything that is happened. It would be better to also create levels of permission, it could be useful to hide knowledge from agents at certain steps.

I decided to go with 3 different classes as the 3 agents in this case are very different: one is the orchestrator, one is a non-intelligent agent and the third is execution agent that can be generalized to other agents. In fact, the code for the chunking agent I left in is extremely similar to the response agent (in production I would make a single class for every intelligent agent).

#### 3.2 Pros and Cons

- The main pro of having an orchestrator is that the system can be easily evolved and scaled. You can add as many agents as you want and let the orchestrator decide whether and when to call the, without having to hard code a workflow;
- Another pro is the flexibility of the workflow, different tasks will result in different workflow, and with an ideal system you can tailor the complexity of the execution to the need of the query, boosting efficiency.

- A con of having an orchestrator is the introduction of non-determinism in the execution workflow. You can be 99% sure that everything will work out well, but you never know whether your system will try to call the response agent before the document retrieval, for example. You can counter it by adding more control to the prompt, but this allows for less reasoning and freedom for the model's intelligence, and adds complexity to the design.
- Another con is that for simple tasks like this one, the orchestrator adds useless complexity, response time and costs.

# That was the brief paper, now there's the README file I would publish if this was a single repo.

# MedTech Regulatory AI Companion: A RAG-based Tool

This code contains the development of a multi-agent AI-powered assistant to navigate the complex landscape of Medical Device regulations. The primary goal is to showcase how Retrieval-Augmented Generation (RAG) can be used to perform a document analysis on FDA and WHO medical de

This project was developed as a proof-of-concept to address the significant regulatory challenges faced by MedTech companies.

---

## Features

In this code you will find a 4-agent approach to document analysis:

* An orchestrator agent
* A chunking agent for semantic chunking
* A non-intelligent RAG agent for information retrieval
* A response agent to tailor the answer

---

## Technology Stack

* **Language:** Python
* **Core AI/RAG Frameworks:** ChromaDB (Vector Store), Sentence-Transformers (Embeddings)
* **LLM Integration:** Google Gemini (`google-generativeai`)
* **PDF Processing:** `pymupdf`
* **Environment Management:** Conda, Pip, `python-dotenv`

---

## Getting Started

Follow these steps to set up and run the project locally.

### 1. Prerequisites

* Python 3.9+
* Conda or another virtual environment manager

### 2. Installation & Setup

1.  **Download the code and documents** 

2.  **Create and activate a Conda environment:**
    ```bash
    conda create -n mdrag_env python=3.9
    conda activate mdrag_env 
    ``` 

3.  **Install libraries:**
    ```bash
    pip install [any library you see imported, I will soon create a requirements.txt file]
    ``` 

4.  **Set up your API Key:**
    * Create a file named `.env` in the root directory of the project.
    * Add your Google Gemini API key to this file:
        ```
        GOOGLE_API_KEY="AIzaSy...your-key-here"
        ```

5.  **Add the Documents:**
    * Create a folder named `documents` in the root directory.
    * Place your files inside this folder.

6.  **Run the code:**
    * Run the code cell by cell. It should work!

In [9]:
# Import libraries

import chromadb
import uuid
import google.generativeai as genai
import os
from dotenv import load_dotenv
import pymupdf4llm
import json
from chromadb.utils import embedding_functions

In [10]:
# Load environmental variables and AI models

load_dotenv() # load the API key and put in a .env file
try:
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
except Exception as e:
    print(f"Error configuring Google AI. Please ensure your API key is correct. Error: {e}")

# Configure paths to data
current_dir = os.getcwd() 
FILE_PATH_FDA_DESIGN = os.path.join(current_dir, '..', 'documents', 'FDA_Design_Control_Guidance.pdf')
FILE_PATH_WHO = os.path.join(current_dir, '..', 'documents', 'WHO_Medical_Device_Regulations.pdf')
FILE_PATH_FDA_POLICY = os.path.join(current_dir, '..', 'documents', 'FDA_Policy_Device_Software_Functions.pdf')
COLLECTION_NAME = "multi_agent_rag"

In [11]:
# Split logic (very simple, splitting paragraphs and skipping short ones)

def split_into_chunks(dict_list):
    text_chunks = []
    metadatas = []
    
    for document in dict_list:
        try:
            paragraphs = list(document.values())[0].split("\n\n")
    
            for paragraph in paragraphs: # Delete short paragraphs
                if len(paragraph) > 10:
                    text_chunks.append(paragraph)
                    metadatas.append({"source": list(document.keys())[0]})
        except Exception as e:
            print(f"An error occurred while processing {list(document.keys())[0]}: {e}")
            
    return text_chunks, metadatas

# Use pymupdf4llm to convert pdf into text, adequately formatted. Using dicts to keep track of sources and add them to metadata while chunking
fda_design_dict = {'FDA_Design_Control_Guidance.pdf': pymupdf4llm.to_markdown(FILE_PATH_FDA_DESIGN)}
who_dict = {'WHO_Medical_Device_Regulations.pdf': pymupdf4llm.to_markdown(FILE_PATH_WHO)}
fda_policy_dict = {'FDA_Policy_Device_Software_Functions.pdf': pymupdf4llm.to_markdown(FILE_PATH_FDA_POLICY)}

text_chunks, metadatas = split_into_chunks([fda_design_dict, who_dict, fda_policy_dict])
print(len(text_chunks), len(metadatas))

"""# Second chunking logic: texting up the documents. I'll convert the pdf in txt and see if the RAG handles them better.
# Result: it worked worse.

# Configure paths to data
current_dir = os.getcwd() 
FILE_PATH_FDA_DESIGN = os.path.join(current_dir, '..', 'documents', 'FDA_Design_Control_Guidance.txt')
FILE_PATH_WHO = os.path.join(current_dir, '..', 'documents', 'WHO_Medical_Device_Regulations.txt')
FILE_PATH_FDA_POLICY = os.path.join(current_dir, '..', 'documents', 'FDA_Policy_Device_Software_Functions.txt')
COLLECTION_NAME = "multi_agent_rag"

def process_txt_file(file_path, source_name):
    all_text_chunks = []
    all_metadatas = []
    try:
        with open(file_path, 'r', encoding = 'utf-8') as f:
            full_text = f.read()

        # Chunking strategy: split by paragraph
        paragraphs = full_text.split('\n\n')

        for para in paragraphs:
            stripped_para = para.strip()
            if len(stripped_para) > 25:  # Filter out very short paragraphs or empty lines
                all_text_chunks.append(stripped_para)
                # Metadata is simpler for a txt file, just the source
                all_metadatas.append({'source': source_name})

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")

    return all_text_chunks, all_metadatas

chunks_fdad, metas_fdad = process_txt_file(FILE_PATH_FDA_DESIGN, 'FDA_Design_Control_Guidance')
print(f"Processed FDA_Design: {len(chunks_fdad)} chunks.")

chunks_who, metas_who = process_txt_file(FILE_PATH_WHO, 'WHO_Medical_Device_Regulations')
print(f"Processed WHO: {len(chunks_who)} chunks.")

chunks_fdap, metas_fdap = process_txt_file(FILE_PATH_FDA_POLICY, 'FDA_Policy_Device_Software_Functions')
print(f"Processed FDA Policy: {len(chunks_fdap)} chunks.")

text_chunks = chunks_fdad + chunks_who + chunks_fdapS
metadatas = metas_fdad + metas_who + metas_fdap"""

# JSON parser function used throughout the code (could become a class method of the agents)
def clean_response(response):
    response_text = response.text
    start_index = response_text.find("{")
    end_index = response_text.rfind("}")
    if start_index != -1 and end_index != -1:
        response_cleaned = json.loads(response_text[start_index : end_index + 1])
        return response_cleaned
    else:
        return response_text

1657 1657


In [12]:
# Define agent classes. Every agent will have a memory variable with a memory limit, and this will be the context that gets passed along
# It would be cleaner to build a 'state' that every agent can access and update (similar to the logic of LangGraph), but this works fine for a POC

class OrchestratorAgent:

    def __init__(self, agents):
        self.agents = agents
        self.memory = []
        self.memory_limit = 15

        self.model = genai.GenerativeModel('gemini-1.5-flash')
        #self.model = genai.GenerativeModel('gemini-1.5-pro-latest')
        #self.model = genai.GenerativeModel('gemini-2.0-flash-thinking-exp-01-21')
        self.generation_config = {
            "temperature": 0.2,
            "top_k": 10,
            "top_p": 0.85,
            "max_output_tokens": 8192
        }

    def run(self): # The main function, where the loop is. Keeps elaborating the input and calling an agent until the user cuts.
        print("Hi! I am Camille, your MedTech regulatory companion. How can I help you?")
        user_input = input("You: ")

        while True:
            self.memory = self.memory[-self.memory_limit:]
            if user_input.lower() in ["exit", "bye", "close"]:
                print("I hope I could be of use to you, have a great day!")
                break  
            orch_response = self.orchestrate(user_input)
            if orch_response["agent_to_call"] == "No action needed":
                print("Is there anything else I can help you with?")
                user_input = input("You: ")
            for agent in self.agents:
                if agent.name == orch_response["agent_to_call"]:
                    print(f"Found agent I was looking for: {agent.name}\n")
                    response = agent.act(orch_response["output"], orch_response["relevant_info"], self.memory)
                    self.memory.append(f"Agent {agent.name} responded {response}")     
        return response

    def orchestrate(self, user_input):
        self.memory.append(f"User: {user_input}")
        self.memory = self.memory[-self.memory_limit:]
        context = "\n".join(self.memory)
        response_format = {"agent_to_call":"", "output": "", "relevant_info":""}
        response = self.model.generate_content(self.get_prompt(context, response_format), generation_config = self.generation_config)
        self.memory.append(f"Orchestrator: {response.text}")
        response_cleaned = clean_response(response)
        return response_cleaned

    def get_prompt(self, context, response_format):
        prompt = f"""
        
        Act as an orchestrator agent for an intelligent RAG system for MedTech companies. 
        
        Your task is to coordinate agents in order to extract relevant documents from a RAG system and package a coherent and precise answer to the query received.
        The task is to call a first time the ragagent, and then use the ragagent documents to call the response_agent on them and craft a response. Make sure to call the response_agent only if the memory contains a previous call to the ragagent.
        
        You will return instructions in a valid JSON in the form of {response_format}. All output should be of string type, the "output" is for the query and the "relevant_info" is to attach documents from the RAG for the response agent.
        
        Your AI agents and their descriptions are {", ".join([f"- {agent.name}: {agent.description}" for agent in self.agents])}
        Use the context, which includes the current user input and the memory of previous inputs and outputs, to plan next steps.
        Context : {context}

        Guidelines:
        At every step, you need to choose only one of the agents and provide instruction to only that agent. If the request needs multiple agent to be solved, do that in a loop.
        Read the context, take your time to understand the task, and check if you have executed it correctly.
        If there are no actions needed, default the "agent_to_call" parameter to "No action needed" in the response.
        Return only the agent name in the "agent_to_call" parameter.
        
        """
        return prompt

class RAGAgent:

    def __init__(self):
        self.name = "ragagent"
        self.description = """I am a RAG agent that can search for relevant documents in a vector database in order to answer a query.
                            I expect a user query as input and will return relevant chunks and a variable containing sources info, relevance scores and chunks.
                            """
        self.memory = []
        self.memory_limit = 15
        self.sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
        self.collection_name = COLLECTION_NAME

    def act(self, query, relevant_info, memory): # Checks whether the db exists before querying it
        if "RAG initialized" not in memory:
            self.collection = self.initialize_db()
            self.memory.append("RAG initialized")
        self.memory.append(memory)
        self.memory = self.memory[-self.memory_limit:]
        documents = self.query_db(query)
        return documents

    def initialize_db(self):
        print("Initializing RAG system... This may take a minute.")
        self.client = chromadb.Client()

        if self.collection_name in [c.name for c in self.client.list_collections()]:
            self.client.delete_collection(name = self.collection_name)
    
        collection = self.client.get_or_create_collection(
        name = self.collection_name,
        embedding_function = self.sentence_transformer_ef
        )
        self.load_documents(collection, text_chunks, metadatas)

        return collection

    def load_documents(self, collection, document_chunks, metadatas):
        collection.add(
        ids = [str(uuid.uuid4()) for _ in text_chunks],
        documents = text_chunks,
        metadatas = metadatas)
    
    def query_db(self, question): # Querys the db retrieving chunks, sources and embedding distances
        results = self.collection.query(query_texts=[question], include = ["documents", "metadatas", "distances"], n_results=10)

        sources_markdown = "### Sources Used for Analysis\n\n"
        retrieved_documents = results['documents'][0]
        retrieved_metadatas = results['metadatas'][0]
        retrieved_distances = results['distances'][0]
    
        for i, (doc, meta, dist) in enumerate(zip(retrieved_documents, retrieved_metadatas, retrieved_distances)):
            # Convert distance to a more intuitive relevance score (1 - distance)
            relevance_score = 1 - dist
            source_info = f"**Source {i+1}:** {meta.get('source', 'N/A')}, Page {meta.get('page', 'N/A')}\n"
            relevance_info = f"**Relevance Score:** {relevance_score:.2f}\n\n"
            content_info = f"```\n{doc}\n```\n\n---\n\n"
            sources_markdown += source_info + relevance_info + content_info
        print(f"Sources, documents and relevance score: {sources_markdown}")
            
        return sources_markdown

class ResponseAgent:
    def __init__(self):
        self.name = "response_agent"
        self.description = """I am a response agent that expects as input a user query and relevant documents and info from a RAG search.
                            My task is to craft a precise response for the user based on the provided documents. I will return a response text.
                            """

        self.memory = []
        self.memory_limit = 15
        self.model = genai.GenerativeModel('gemini-1.5-flash')
        #self.model = genai.GenerativeModel('gemini-1.5-pro-latest')
        #self.model = genai.GenerativeModel('gemini-2.0-flash-thinking-exp-01-21')
        self.generation_config = {
            "temperature": 0.2,
            "top_k": 10,
            "top_p": 0.85,
            "max_output_tokens": 8192
        }
    
    def act(self, query, relevant_info, memory): # Gets the prompt, generate the response and prints it
        prompt = self.get_prompt(query, relevant_info, memory)
        response = self.model.generate_content(prompt, generation_config = self.generation_config)
        print(response.text)
        return response.text

    def get_prompt(self, query, relevant_info, memory):
        self.memory = self.memory[-self.memory_limit:]
        prompt = f"""
        
        Act as Camille, an AI companion acting as a Senior Consultant for medical devices. 
        
        You receive a query from your client, and your task is to answer to it based on the relevant information you receive from the RAG system as document chunks.

        Cite the documents and sources you receive as part of your input. The structure of your answer will be:
        - Salutation.
        - Precise response to the query based on the documents received from the RAG.
        - Strategic recommendation

        As additional resources and context:
        User input: {query}
        Relevant documents with sources and relevance scores: {relevant_info}
        Memory of previous inputs and info: {memory}

        Guidelines:
        Be precise, confident and do not make things up. If the context is not deatiled enough to provide a clear answer, state it.

        """
        return prompt

In [ ]:
# Added an agent to chunk documents. It works well but I couldn't end the parsing in time, therefore I didn't add it.
# I have used the response_schema on this one, worked decently but not as good as expected.

class ChunkingAgent():
    def __init__(self):
        self.name = "chunking_agent"
        self.description = """I am an LLM agent created for semantic chunking. My task is to receive pdf documents and return a list of semantically coherent chunks.
                            """

        self.memory = []
        self.memory_limit = 15
        self.text_chunks = []
        self.metadatas = []
        self.model = genai.GenerativeModel('gemini-2.5-pro-preview-03-25')
        self.generation_config = {
            "temperature": 0.2,
            "top_k": 10,
            "top_p": 0.85,
            "response_mime_type": "application/json",
            "response_json_schema": {"chunks": list}
        }
    
    def act(self, query, relevant_info, memory):
        fda_design_dict = {'FDA_Design_Control_Guidance.pdf': pymupdf4llm.to_markdown(FILE_PATH_FDA_DESIGN)}
        who_dict = {'WHO_Medical_Device_Regulations.pdf': pymupdf4llm.to_markdown(FILE_PATH_WHO)}
        fda_policy_dict = {'FDA_Policy_Device_Software_Functions.pdf': pymupdf4llm.to_markdown(FILE_PATH_FDA_POLICY)}
        documents = [fda_design_dict, who_dict, fda_policy_dict]

        for document in documents:
            source = list(document.keys())[0]
            cleaned_chunks = self.chunk_document(list(document.values())[0])
            self.text_chunks.append(cleaned_chunks)
            for i in range(len(cleaned_chunks)):
                self.metadatas.append({"source": source})
        print(f"metadatas: {self.metadatas}")
        return [self.text_chunks, self.metadatas]

    def chunk_document(self, document):
        cleaned_chunks = []
        max_length = 10000
        for i in range(0, len(document), max_length):
            text_chunk = document[i : i + max_length]
            prompt = self.get_prompt(text_chunk)
            chunks = self.model.generate_content(prompt)
            cleaned_part = clean_response(chunks)["chunks"]
            cleaned_chunks.extend(cleaned_part)
        return cleaned_chunks
        
    def get_prompt(self, document):
        #response_format = {"chunks": []}
        # You MUST return a valid JSON in the form of {response_format}, where in "chunks" is a list where each item is a single, coherent, chunk of text from the document.
        prompt = f"""
        
        Act as an expert in MedTech regulations. 
        
        You receive a document about medical device regulations and  your task is to divide it into semantically coherent chunks for a RAG system.
        
        Make sure to open and close the brackets for the JSON and the list correctly. Make sure to escape possible double quotes in the text.
        
        Documents to chunk:
        {document}

        Guidelines:

        Straightforward techniques involve separating paragraph, accounting for end of sentences indicators like '.', '!', '?' and so on.
        Make your own techniques using multi-step reasoning (ri-validate your chunks if necessary). Avoid splitting sentences or ideas across different chunks.
        Chunks need to be semantically coherent. Each chunk should represent a complete thought, topic, or regulatory requirement.
        
        """
        return prompt

In [13]:
# Instantiate agent objects and run

ragagent = RAGAgent()
response_agent = ResponseAgent()
orchestrator = OrchestratorAgent([ragagent, response_agent])

orchestrator.run()

Hi! I am Camille, your MedTech regulatory companion. How can I help you?


You:  What documentation is needed for a mobile app that  monitors heart rate?


Found agent I was looking for: ragagent

Initializing RAG system... This may take a minute.
Sources, documents and relevance score: ### Sources Used for Analysis

**Source 1:** FDA_Policy_Device_Software_Functions.pdf, Page N/A
**Relevance Score:** 0.39

```

FDA has cleared several mobile medical apps with attachments to a mobile
platform. Specifically, patient monitoring mobile apps that monitor a patient
for heart rate variability from a signal produced by an electrocardiograph,
vectorcardiograph, or blood pressure monitor are classified as cardiac
monitoring software under 21 CFR 870.2300 (Cardiac monitor (including
```

---

**Source 2:** FDA_Policy_Device_Software_Functions.pdf, Page N/A
**Relevance Score:** 0.26

```

cardiotachometer and rate alarm)). Other mobile medical apps that use a
hardware attachment or interface to a monitoring system that have been
cleared include an automatic electronic blood pressure monitor under 21 CFR
870.1130 and a perinatal monitoring system und

You:  Is our AI-powered MRI analysis tool considered a medical  device software?


Found agent I was looking for: ragagent

Initializing RAG system... This may take a minute.
Sources, documents and relevance score: ### Sources Used for Analysis

**Source 1:** FDA_Policy_Device_Software_Functions.pdf, Page N/A
**Relevance Score:** 0.12

```

**Answer** : Software used in the production process for medical devices, or for collecting,
storing and maintaining quality system data collection for medical devices (including
complaint submissions) is not considered a medical device on its own. This software does not
meet the definition of medical device, but is part of the quality system. However this
software is required to comply with the appropriate good manufacturing practices (GMP)
regulations. [94]
```

---

**Source 2:** FDA_Policy_Device_Software_Functions.pdf, Page N/A
**Relevance Score:** 0.12

```

FDA has previously clarified that when a software application is used to analyze medical
device data, it has traditionally been regulated as an accessory to a medical de

You:  exit


I hope I could be of use to you, have a great day!


"Dear [Client Name],\n\nBased on the provided FDA documentation (Sources 1, 2, 7, 8, 9), your AI-powered MRI analysis tool is *likely* considered medical device software.  Source 2 explicitly states that software analyzing medical device data is traditionally regulated as medical device software or an accessory to a medical device.  Source 7 provides an example of software analyzing imaging findings for clinical decision-making, which is analogous to your MRI analysis tool.  Source 8 emphasizes that software transforming a general-purpose platform into a regulated medical device is considered device software.  Finally, Source 9 gives examples of software analyzing medical data to detect critical conditions, further supporting the classification of your tool as medical device software.\n\nHowever, the precise classification depends on the *specific functionality* of your tool. If it simply displays MRI data without interpretation or clinical decision support, the regulatory requirements